## Make connection

In [1]:
import paramiko, os, re
from getpass import getpass

# Create the passwd prompt
thePass = getpass(prompt="Please enter your SSH Password: ")


# Host Information
host = "192.168.6.71"
port = 2222
username = "paul.gleason"
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication Failed")

## Check Directory

In [2]:
stdin, stdout, stderr = ssh.exec_command('pwd')
print(stdout.readlines())

['/home/paul.gleason\n']


## Place kraken file and verify it's in the right location with ls

In [3]:
sftp = ssh.open_sftp()

ssh.exec_command('cd /home/paul.gleason/')

sftp.put('kraken', 'kraken') 

stdin, stdout, stderr = ssh.exec_command('ls')
print(stdout.readlines())


['kraken\n']


## Run kraken file and write output to kraken.txt

In [9]:
if os.path.exists('kraken.txt'):
    usrinput = input("Would you like to delete this file? (Y/N) ")
    if usrinput == "Y" or usrinput == "y":
        os.remove("kraken.txt")

thePass = getpass(prompt="Please enter your user Password: ")

stdin, stdout, stderr = ssh.exec_command("sudo -S /home/paul.gleason/kraken --folder /usr/bin --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin")

stdin.write(thePass + "\n")
stdin.flush()

# for line in stdout.readlines():
#     with open('pg.pid', 'a') as f:
#         f.write(line)

for line in stdout.readlines():
    if re.findall(r'.*pid.*', line):
        with open('pid.txt', 'a') as f:
            f.write(line.split(" ")[-2].split("=")[-1])


519423
561559


## lsof -p XXXX

## Check "bin" directories for malicious files

In [57]:
cmds = ["ls -al -h k /usr/bin/", "ls -al -h k /usr/sbin", "ls -al -h k /usr/local/bin", "ls -al -h k /usr/local/sbin"]

if os.path.exists('bin_results.txt'):
    usrinput = input("Would you like to delete this file? (Y/N) ")
    if usrinput == "Y" or usrinput == "y":
        os.remove("bin_results.txt")

for command in cmds:
    # Get the outpt form the command
    stdin, stdout, stderr = ssh.exec_command(command)

    split_lines = []

    for line in stdout.readlines():
        # print(line.split(' '))
        split_lines.append(line.split(' '))
        
        # Searches and returns results using a regular expression search

    files = []
    split_command = command.split(' ')

    # print(split_lines)
    # loop through split_lines
    for new_line in split_lines:
        # if new_line is longer than 2
        if len(new_line) > 2:
            # if there is a symlink
            if new_line[-2] != '->':
                for item in new_line:
                    # print(item)
                    if '\n' in item:
                        # print(item[0:-2])
                        files.append(item[0:-2])
                        
            else:
                # print(new_line[-3])
                files.append(new_line[-3])

    # for r in files:
    #     print(r)


    for file in files:
        x = re.findall(r'^([a-z]{6,17})$', file)
        for found in x:
            output = split_command[-1] + found

    
    # Get results from stdout
    # lines = stdout.readlines()
    # print(lines)

    # Cover the list to a string
    # output = ''.join(lines)

            # Header output
            sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'
            # Footer
            sepFooter = '' + '### END ' + command + ' ###\n\n'

            # Concatenate the header, output, and footer
            cmd_output = sepHeader + output + sepFooter

            # Sace the cmd_output to a file
            # a is append mode
            with open('bin_results.txt', 'a') as f:
                f.write(output + '\n')

## Run lsof using sudo

In [4]:
thePass = getpass(prompt="Please enter your user Password: ")

stdin, stdout, stderr = ssh.exec_command("sudo -S lsof -i -n | awk '{print $2}'")
stdin.write(thePass + "\n")
stdin.flush()

for line in stdout.readlines():
    for _ in range(1):
        print(line)

PID

593

603

603

830

855

855

935

935

561559

600020

600170

600390

600489

600523

600626

600716

600818

600854

600947

600952

601052

601067

601169

601242

601357

601383

601485

601534

601633

601653

601752

601759

601861

601881

601977

601982

602084

602108

602204

602220

602318

602330

602429

602470

602568

602604

602705

602900

603028

603036

603134

603758

603871

604366

604465



## LS p.datlon Folder

In [7]:
stdin, stdout, stderr = ssh.exec_command("ls /home/p.dalton")

for line in stdout.readlines():
    print(line)

aqwerpuqwerj

d.pid

test-lk



## Close connection

In [5]:
ssh.close()

## Reflections

### Based on what you discovered and the information in the threat report, what is your conclusion about the host you analyzed?

### What did you like the most and least about this assignment?

### What additional questions do you have?
